# Referencing Partial State Update Blocks labels to substeps in cadCAD

*Vitor Marthendal Nunes*

---

This notebook shows how to use label metadata on PSUBs to do post-processing on the simulation. We use the key `label` as metadata to the partial state update blocks, so it's possible to map the substeps order to the PSUBs label. The used prey and predator model was taken from `minimal_prey_predator.ipynb`

## Dependences

In [1]:
%%capture
!pip install cadcad 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor

## Definitions

### Initial conditions and parameters

In [3]:
initial_conditions = {
    'prey_population': 100,
    'predator_population': 15
    }

params = {
    "prey_birth_rate": [1.0],
    "predator_birth_rate": [0.01],
    "predator_death_const": [1.0],
    "prey_death_const": [0.03],
    "dt": [0.1] # Precision of the simulation. Lower is more accurate / slower
}

simulation_parameters = {
    'N': 1,
    'T': range(200),
    'M': params
}

### Policies

In [4]:
def p_predator_births(params, step, sL, s):
  dt = params['dt']
  predator_population = s['predator_population']
  prey_population = s['prey_population']
  birth_fraction = params['predator_birth_rate'] + np.random.random() * 0.0002
  births =  birth_fraction * prey_population * predator_population * dt
  return {'add_to_predator_population': births}


def p_prey_births(params, step, sL, s):
  dt = params['dt']
  population = s['prey_population']
  birth_fraction = params['prey_birth_rate'] + np.random.random() * 0.1
  births =  birth_fraction * population * dt
  return {'add_to_prey_population': births}


def p_predator_deaths(params, step, sL, s):
  dt = params['dt']
  population = s['predator_population']
  death_rate = params['predator_death_const'] + np.random.random() * 0.005
  deaths = death_rate * population * dt
  return {'add_to_predator_population': -1.0 * deaths}


def p_prey_deaths(params, step, sL, s):
  dt = params['dt']
  death_rate = params['prey_death_const'] + np.random.random() * 0.1
  prey_population = s['prey_population']
  predator_population = s['predator_population']
  deaths = death_rate * prey_population * predator_population * dt
  return {'add_to_prey_population': -1.0 * deaths}

### State update functions

In [5]:
def s_prey_population(params, step, sL, s, _input):
    y = 'prey_population'
    x = s['prey_population'] + _input['add_to_prey_population']
    return (y, x)


def s_predator_population(params, step, sL, s, _input):
    y = 'predator_population'
    x = s['predator_population'] + _input['add_to_predator_population']
    return (y, x)

### State update blocks

In [6]:
partial_state_update_blocks = [
    { 
        'label': 'Predator dynamics',
        'policies': {
            'predator_births': p_predator_births,
            'predator_deaths': p_predator_deaths
        },
        'variables': {
            'predator_population': s_predator_population            
        }
    },
    {
        'label': 'Prey dynamics',
        'policies': {
            'prey_births': p_prey_births,
            'prey_deaths': p_prey_deaths
        },
        'variables': {
            'prey_population': s_prey_population
        }
    }
]

### Configuration and Execution

In [7]:
sim_config = config_sim(simulation_parameters)

exp = Experiment()
exp.append_configs(sim_configs=sim_config, 
                   initial_state=initial_conditions,
                   partial_state_update_blocks=partial_state_update_blocks)


from cadCAD import configs
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.local_mode)
executor = Executor(exec_context=exec_context, configs=configs) 
(records, tensor_field, _) = executor.execute() 


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (200, 5, 1, 2)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.10s


### Results

In [8]:
df = pd.DataFrame(records)
df

,prey_population,predator_population,simulation,subset,run,substep,timestep
0,100.000000,15.000000,0,0,1,0,0
1,100.000000,15.004699,0,0,1,1,1
2,102.742647,15.004699,0,0,1,2,1
3,102.742647,15.063790,0,0,1,1,2
4,93.551318,15.063790,0,0,1,2,2
...,...,...,...,...,...,...,...
396,76.896687,16.597650,0,0,1,2,198
397,76.896687,16.207445,0,0,1,1,199
398,76.636509,16.207445,0,0,1,2,199
399,76.636509,15.843644,0,0,1,1,200


In [9]:
# Mapping the substep order to the PSUB label
psubs = partial_state_update_blocks
psub_map = {order+1: psub['label'] for (order, psub) in enumerate(psubs)}

In [10]:
df['psubs'] = df.substep.map(psub_map)
df

,prey_population,predator_population,simulation,subset,run,substep,timestep,psubs
0,100.000000,15.000000,0,0,1,0,0,NaN
1,100.000000,15.004699,0,0,1,1,1,Predator dynamics
2,102.742647,15.004699,0,0,1,2,1,Prey dynamics
3,102.742647,15.063790,0,0,1,1,2,Predator dynamics
4,93.551318,15.063790,0,0,1,2,2,Prey dynamics
...,...,...,...,...,...,...,...,...
396,76.896687,16.597650,0,0,1,2,198,Prey dynamics
397,76.896687,16.207445,0,0,1,1,199,Predator dynamics
398,76.636509,16.207445,0,0,1,2,199,Prey dynamics
399,76.636509,15.843644,0,0,1,1,200,Predator dynamics


### Filtering the results by the PSUB labels

In [11]:
df.query("psubs=='Predator dynamics'")

,prey_population,predator_population,simulation,subset,run,substep,timestep,psubs
1,100.000000,15.004699,0,0,1,1,1,Predator dynamics
3,102.742647,15.063790,0,0,1,1,2,Predator dynamics
5,93.551318,14.972797,0,0,1,1,3,Predator dynamics
7,91.912942,14.872089,0,0,1,1,4,Predator dynamics
9,92.356510,14.769655,0,0,1,1,5,Predator dynamics
...,...,...,...,...,...,...,...,...
391,76.972421,17.557499,0,0,1,1,196,Predator dynamics
393,69.580180,17.033520,0,0,1,1,197,Predator dynamics
395,73.256779,16.597650,0,0,1,1,198,Predator dynamics
397,76.896687,16.207445,0,0,1,1,199,Predator dynamics


In [12]:
df.query("psubs=='Prey dynamics'")

,prey_population,predator_population,simulation,subset,run,substep,timestep,psubs
2,102.742647,15.004699,0,0,1,2,1,Prey dynamics
4,93.551318,15.063790,0,0,1,2,2,Prey dynamics
6,91.912942,14.972797,0,0,1,2,3,Prey dynamics
8,92.356510,14.872089,0,0,1,2,4,Prey dynamics
10,95.204167,14.769655,0,0,1,2,5,Prey dynamics
...,...,...,...,...,...,...,...,...
392,69.580180,17.557499,0,0,1,2,196,Prey dynamics
394,73.256779,17.033520,0,0,1,2,197,Prey dynamics
396,76.896687,16.597650,0,0,1,2,198,Prey dynamics
398,76.636509,16.207445,0,0,1,2,199,Prey dynamics
